In [ ]:
import pandas as pd 
#https://www.census.gov/data/datasets/time-series/demo/sahie/estimates-acs.html

In [ ]:

data = pd.read_csv('sahie_2022.csv', low_memory=False)
data['state_name'] = data['state_name'].str.strip() # remove the spaces 
data['county_name'] = data['county_name'].str.strip()# remove the spaces 
texas_data = data[data['state_name'] == 'Texas'].copy() #only Texas data
texas_data.drop(columns='Unnamed: 25', inplace=True) #remove blank column on right 
texas_data.dropna(axis=0, inplace=True) #remove rows with Na values 
texas_data.drop(columns=['NIPR','nipr_moe', 'PCTUI', 'pctui_moe', 'PCTIC', 
                         'pctic_moe', 'sexcat', 
                         'statefips', 'countyfips', 'geocat',
                         'version'], inplace=True) #keep columns that I want 
texas_data = texas_data[texas_data['county_name'] != ''].copy() #remove rows with a blank county name 
texas_data.reset_index(drop=True, inplace=True)
texas_data.rename(columns={'iprcat':'Income Category', 'PCTELIG':'Percent Uninsured for all income levels', 
                           'NUI':'Number Uninsured',
                           'NIC': 'Number Insured', 'nui_moe': 'Number Uninsured MOE', 
                           'nic_moe':'Number Insured MOE',
                           'pctelig_moe':'Percent Uninsured for all income levels MOE', 
                           'PCTLIIC':'Percent Insured for all income levels',
                           'pctliic_moe':'Percent Insured for all income levels MOE'}, inplace=True)
texas_data

,year,agecat,racecat,Income Category,Number Uninsured,Number Uninsured MOE,Number Insured,Number Insured MOE,Percent Uninsured for all income levels,Percent Uninsured for all income levels MOE,Percent Insured for all income levels,Percent Insured for all income levels MOE,state_name,county_name
0,2022,0,0,0,6314,659,29440,659,17.7,1.8,82.3,1.8,Texas,Anderson County
1,2022,0,0,1,3237,433,12015,567,9.1,1.2,33.6,1.6,Texas,Anderson County
2,2022,0,0,2,3922,492,14750,613,11.0,1.4,41.3,1.7,Texas,Anderson County
3,2022,0,0,3,2195,325,7802,476,6.1,0.9,21.8,1.3,Texas,Anderson County
4,2022,0,0,4,5332,589,21458,680,14.9,1.6,60.0,1.9,Texas,Anderson County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24379,2022,5,0,1,291,72,896,90,12.2,3.0,37.4,3.8,Texas,Zavala County
24380,2022,5,0,2,332,78,1099,93,13.9,3.3,45.9,3.9,Texas,Zavala County
24381,2022,5,0,3,216,57,610,80,9.0,2.4,25.5,3.3,Texas,Zavala County
24382,2022,5,0,4,409,88,1586,98,17.1,3.7,66.2,4.1,Texas,Zavala County
